##### Jupyter Notebook is available at : [Batch Docking](https://github.com/MangalamGSinha/DockSuiteX/tree/main/docs/examples/batch_docking.ipynb)

In [ ]:
from docksuitex.batch_docking import BatchProtein, BatchLigand, BatchPocketFinder, BatchVinaDocking, BatchAD4Docking
from docksuitex.utils import fetch_pdb, fetch_sdf, view_molecule, parse_vina_log_to_csv, parse_ad4_dlg_to_csv

### Fetch and prepare Protein

In [ ]:
# Fetch multiple protein structures (1HVR, 2TRX) in parallel
fetch_pdb(pdbid = ["1HVR", "2TRX"], save_to="proteins", parallel=2)

In [ ]:
# Initialize BatchProtein to process all proteins in the 'proteins' directory
# This cleans the PDB, removes water/heterogens, and adds hydrogens/charges

batch_prot = BatchProtein(
    inputs="proteins",

    #Default parameters
    fix_pdb=True,
    remove_heterogens=True,
    remove_water=True,
    add_hydrogens=True,
    add_charges=True,
    preserve_charge_types=None, #eg: ["Zn", "Fe"]
)


# Run preparation for all proteins using 8 CPUs
batch_prot.prepare_all(save_to="prepared_proteins", cpu = 8)

### Find Binding Pockets

In [ ]:
# Identify binding pockets for all prepared proteins using P2Rank
batch_pf = BatchPocketFinder(inputs="prepared_proteins",  max_centres=3)

# Run pocket detection and return receptors with their pocket centers
receptors_with_centers = batch_pf.run_all(save_to="pockets_output", cpu=8)

### Fetch and prepare Ligand

In [ ]:
# Fetch multiple ligand structures in parallel
fetch_sdf(cid =  ["228", "338"], save_to="ligands", parallel=2)

In [ ]:
# Initialize BatchLigand to process all ligands in the 'ligands' directory
# This minimizes the structure, removes water and adds hydrogens/charges

batch_lig = BatchLigand(
    inputs="ligands",

    #Default parameters
    minimize=None,  #Options: "mmff94", "mmff94s", "uff", "gaff"
    remove_water=True,
    add_hydrogens=True,
    add_charges=True,
    preserve_charge_types=None,
)
# Run preparation for all ligands using 8 CPUs
batch_lig.prepare_all(save_to="prepared_ligands", cpu=8)

### Dock each ligand against a each receptor at all the predicted protein pockets

##### Using Vina

In [ ]:
# Initialize BatchVinaDocking with prepared receptors (and their pockets) and ligands
batch_vina = BatchVinaDocking(
    receptors_with_centers=receptors_with_centers,
    ligands="prepared_ligands",
    seed=42, # Random seed for reproducibility

    # Default Parameters
    grid_size=(20,20,20),
    exhaustiveness=8,
    num_modes=9, 
)

# Run Vina docking for all receptor-ligand pairs using 8 CPUs
batch_vina.run_all(save_to="vina_batch_results", cpu=8)

# Parse all Vina output logs into a single CSV summary
batch_vina.parse_results()

##### Using AutoDock4

In [ ]:
# Initialize BatchAD4Docking with prepared receptors (and their pockets) and ligands
batch_ad4 = BatchAD4Docking(
    receptors_with_centers=receptors_with_centers,
    ligands="prepared_ligands",
    seed=(21, 42),  #Random seed for reproducibility

    #Default Parameters
    grid_size=(60,60,60),
    spacing=0.375,     
    dielectric=-0.1465, 
    smooth=0.5,          
    ga_pop_size=150,
    ga_num_evals=2500000,
    ga_num_generations=27000,
    ga_elitism=1,
    ga_mutation_rate=0.02,
    ga_crossover_rate=0.8,
    ga_run=10,
    rmstol=2.0,
)

# Run AutoDock4 docking for all receptor-ligand pairs using 8 CPUs
results_ad4 = batch_ad4.run_all(save_to="ad4_batch_results", cpu=8)

# Parse all AD4 output logs into a single CSV summary
batch_ad4.parse_results()